<a href="https://colab.research.google.com/github/tanvik7/Binance-Trade/blob/main/Trade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing the dependencies
import pandas as pd
import numpy as np
import ast

In [4]:
# loading the dataset to a Pandas DataFrame, handling potential errors
try:
    data = pd.read_csv('/content/TRADES_CopyTr_90D_ROI.csv')  # Try reading the file normally
except pd.errors.ParserError:
    data = pd.read_csv('/content/TRADES_CopyTr_90D_ROI.csv', error_bad_lines=False)  # Skip bad lines if found
    print("Warning: Some lines were skipped due to parsing errors.")  # Alert the user about skipped lines
    # Further investigation might be needed to identify and fix the bad lines
    # You can examine the skipped lines using pd.errors.ParserError.lines property
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [5]:
#dataset info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


In [6]:
#first 5 rows of the data
data.head()

,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."


In [7]:
#last 5 rows of data
data.tail()

,Port_IDs,Trade_History
145,4000222729738650369,"[{'time': 1718982068000, 'symbol': 'ARKMUSDT',..."
146,3998659472131949824,"[{'time': 1718979385000, 'symbol': 'BTCUSDT', ..."
147,4028701921959171840,"[{'time': 1718984241000, 'symbol': 'BTCUSDT', ..."
148,4014818740371615232,"[{'time': 1718983357000, 'symbol': 'SOLUSDT', ..."
149,3768170840939476993,"[{'time': 1718977395000, 'symbol': 'BNBUSDT', ..."


In [8]:
# checking the number of missing values in each column
data.isnull().sum()

,0
Port_IDs,0
Trade_History,1


In [9]:
#handling missing values
data.dropna(inplace=True)


In [10]:
#re checking the dataset for missing values
print(data.isnull().sum())


Port_IDs         0
Trade_History    0
dtype: int64


In [11]:
#grouping the dataset by port id
accounts = data.groupby('Port_IDs')

In [12]:
#Converting JSON string to list of dictionaries
data['Trade_History'] = data['Trade_History'].apply(ast.literal_eval)

In [13]:
# Access first trade
print(data['Trade_History'][0][0]['symbol'])

SOLUSDT


In [14]:
# Function to extract all values from a specific key in Trade_History
def extract_key_values(data, key):
    values = []
    for trades in data['Trade_History']:  # Iterate over all trade history lists
        for trade in trades:  # Iterate over individual trade dictionaries
            if key in trade:
                values.append(trade[key])  # Collect the value of the key
    return values

In [15]:
#Extracting  all  values required for calculation
r=extract_key_values(data , 'realizedProfit')
q=extract_key_values(data , 'quantity')


In [16]:
#function to calculate all the metrices
def calculate_metrics(account_data):
    # Extract 'realizedProfit' and 'quantity' for the current group
    r = extract_key_values(account_data, 'realizedProfit')
    q = extract_key_values(account_data, 'quantity')

    total_pnl = sum(r)  # Calculate total PnL using the extracted values
    initial_investment = sum(q)  # Calculate initial investment using the extracted values
    roi = (total_pnl / initial_investment) * 100 if initial_investment != 0 else 0

    # Sharpe Ratio Calculation
    pnl_series = pd.Series(r)  # Create a Series for PnL calculations
    sharpe_ratio = pnl_series.mean() / pnl_series.std() if pnl_series.std() != 0 else 0

    # Maximum Drawdown Calculation
    cumulative_pnl = pnl_series.cumsum()
    running_max = cumulative_pnl.cummax()
    drawdown = running_max - cumulative_pnl
    max_drawdown = drawdown.max()

    # Win Rate & Position Counts
    total_positions = len(r)  # Use the length of 'r' for total positions
    win_positions = sum(1 for profit in r if profit > 0)  # Count winning positions
    win_rate = (win_positions / total_positions) * 100 if total_positions != 0 else 0

    return {
        'PnL': total_pnl,
        'ROI': roi,
        'Sharpe_Ratio': sharpe_ratio,
        'Max_Drawdown': max_drawdown,
        'Win_Rate': win_rate,
        'Win_Positions': win_positions,
        'Total_Positions': total_positions
    }

# Compute metrics for each account
metrics = accounts.apply(calculate_metrics).apply(pd.Series)


<ipython-input-16-129bf4a9706c>:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = accounts.apply(calculate_metrics).apply(pd.Series)


In [17]:
# Normalize and Rank Accounts
metrics['Score'] = (
    metrics['ROI'] * 0.4 +
    metrics['Sharpe_Ratio'] * 0.3 +
    (100 - metrics['Max_Drawdown']) * 0.2 +
    metrics['Win_Rate'] * 0.1
)
metrics = metrics.sort_values(by='Score', ascending=False)

In [18]:
# Save results to CSV
metrics.to_csv('account_metrics.csv')


In [19]:
# Get Top 20 Accounts
top_20 = metrics.head(20)
print(top_20)


                              PnL        ROI  Sharpe_Ratio  Max_Drawdown  \
Port_IDs                                                                   
3826087012661391104    532.656974  12.178517      0.675101     12.925980   
3768170840939476993    243.668899   8.779089      0.424277      0.000000   
3986814617275053313  16337.461881   0.871628      0.329787      2.381947   
3956048468100538880   1373.564890   1.114561      0.524862      0.000000   
3891020560590657281   2856.300564   1.743407      0.432477      0.000000   
4039279455324236544   1038.807419   1.022101      0.471684      0.000000   
4017110277719148289   2899.933021   0.471231      0.394026      0.000000   
4039129759104249600   1264.289200   3.162479      0.522043      0.000000   
4017323550554338817   3134.735103   1.957286      0.213208      0.000000   
4035430878731345664   2493.750420   1.909902      0.469639      0.000000   
3977234346014419201   2427.292831   4.102464      0.304736      0.000000   
402974987168

In [20]:
#saving the top 20 in a csv file
top_20.to_csv('top_20_accounts.csv')